In [16]:
import duckdb
from os import remove

SF = 1

duckdb.__version__

'v0.7.1-dev648'

In [14]:
database_file = "snb.duckdb"

con = duckdb.connect(database_file)
schema = open(f"SNB-SF{SF}-dataset/schema.sql").read().split(";")
for query in schema:
    con.execute(query)
load = open(f"SNB-SF{SF}-dataset/load.sql").read().split(";")
for query in load:
    con.execute(query)
print(f"Loaded dataset of scale factor {SF}")

Loaded dataset of scale factor 1


In [15]:
con.sql("""load 'sqlpgq'""")

In [ ]:
con.sql("""CREATE PROPERTY GRAPH social_network
    VERTEX TABLES (
        Person PROPERTIES (id, firstName, lastName) LABEL Person,

        University PROPERTIES ARE ALL COLUMNS EXCEPT (type) LABEL University,

        Tag LABEL Tag
    )
    EDGE TABLES (
       Person_studyAt_University 
            SOURCE KEY (PersonId) REFERENCES Person (id)
            DESTINATION KEY (UniversityId) REFERENCES University (id)
            PROPERTIES (PersonId, UniversityId, classYear)
            LABEL Person_studyAt_University,

        Person_hasInterest_Tag 
            SOURCE KEY (PersonId) REFERENCES Person (id)
            DESTINATION KEY (TagId) REFERENCES Tag (id) 
            PROPERTIES ARE ALL COLUMNS EXCEPT (creationDate)
            LABEL Person_hasInterest_Tag,

        Person_knows_Person 
            SOURCE KEY (Person1id) REFERENCES Person (id)
            DESTINATION KEY (Person2id) REFERENCES Person (id)
            LABEL Person_knows_Person
    )""")

In [ ]:
con.sql("""
    SELECT pg.bob, pg.bob_id, count(pg.friend_name) as number_of_friends

    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person WHERE p.firstName = 'Bob')-[k:Person_knows_Person]->(p2:person)
    COLUMNS (p.firstName as bob, p.id as bob_id, p2.firstName as friend_name) 
    ) pg

    GROUP BY pg.bob, pg.bob_id
""").fetchdf()

In [ ]:
print(con.sql("""
    SELECT pg.bob, pg.bob_id, count(pg.friend_name) as number_of_friends

    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person WHERE p.firstName = 'Bob')-[k:Person_knows_Person]->(p2:person)
    COLUMNS (p.firstName as bob, p.id as bob_id, p2.firstName as friend_name) 
    ) pg

    GROUP BY pg.bob, pg.bob_id
""").explain())


In [ ]:
print(con.sql("""
    SELECT pg.firstname, pg.id, count(pg.reachable_people)
    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person)-[k:Person_knows_Person]->{1,4}(p2:person)
    WHERE p.firstname = 'Bob'
    COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
    ) pg
    GROUP BY ALL
""").explain())


In [ ]:
EXPLAIN SELECT pg.firstname, pg.id, count(pg.reachable_people)
FROM GRAPH_TABLE (social_network,
MATCH (p:Person)-[k:Person_knows_Person]->{1,4}(p2:person)
WHERE p.firstname = 'Bob'
COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
) pg
GROUP BY ALL;

In [17]:
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [18]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///snb.duckdb

%sql load 'sqlpgq'

""


In [19]:
%%sql
    SELECT pg.firstname, pg.id, count(pg.reachable_people)
    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person)-[k:Person_knows_Person]->{1,4}(p2:person)
    WHERE p.firstname = 'Bob'
    COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
    ) pg
    GROUP BY ALL

(duckdb.ParserException) Parser Error: syntax error at or near "1"
LINE 3: ...g.firstname, pg.id, count(pg.reachable_people)
    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person)-[k:Person_knows_Person]->(1, 4)(p2:person)
                                                  ^
[SQL: SELECT pg.firstname, pg.id, count(pg.reachable_people)
    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person)-[k:Person_knows_Person]->(1, 4)(p2:person)
    WHERE p.firstname = 'Bob'
    COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
    ) pg
    GROUP BY ALL]
(Background on this error at: https://sqlalche.me/e/14/f405)


/Users/dljtw/git/duckdb-pgq/.venv/lib/python3.10/site-packages/sql/command.py:109: FutureWarning: Variable substitution with $var and {var} has been deprecated and will be removed in a future version. Use {{var}} instead. To remove this, see: https://jupysql.ploomber.io/en/latest/howto.html#ignore-deprecation-warnings
  warnings.warn(msg, FutureWarning)


In [ ]:
%sql select {1,4} from person limit 10

In [ ]:
%%sql
            SELECT pg.firstname, pg.id, count(pg.reachable_people)
            FROM GRAPH_TABLE (social_network,
            MATCH (p:Person)-[k:Person_knows_Person]-> +(p2:person)
            WHERE p.firstname = 'Bob'
            COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
            ) pg
            GROUP BY ALL

In [ ]:
%%sql
            SELECT pg.firstname, pg.id, count(pg.reachable_people)
            FROM GRAPH_TABLE (social_network,
            MATCH (p:Person)-[k:Person_knows_Person]-> *(p2:person)
            WHERE p.firstname = 'Bob'
            COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
            ) pg
            GROUP BY ALL

In [ ]:
%%sql
    CREATE PROPERTY GRAPH social_network
    VERTEX TABLES (
        Person PROPERTIES (id, firstName, lastName) LABEL Person,

        University PROPERTIES ARE ALL COLUMNS EXCEPT (type) LABEL University,

        Tag LABEL Tag
    )
    EDGE TABLES (
       Person_studyAt_University 
            SOURCE KEY (PersonId) REFERENCES Person (id)
            DESTINATION KEY (UniversityId) REFERENCES University (id)
            PROPERTIES (PersonId, UniversityId, classYear)
            LABEL Person_studyAt_University,

        Person_hasInterest_Tag 
            SOURCE KEY (PersonId) REFERENCES Person (id)
            DESTINATION KEY (TagId) REFERENCES Tag (id) 
            PROPERTIES ARE ALL COLUMNS EXCEPT (creationDate)
            LABEL Person_hasInterest_Tag,

        Person_knows_Person 
            SOURCE KEY (Person1id) REFERENCES Person (id)
            DESTINATION KEY (Person2id) REFERENCES Person (id)
            LABEL Person_knows_Person
    )

In [ ]:
%%sql
    SELECT pg.p1name, pg.p2name
    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person)-[k:Person_knows_Person]->(p2:person)-[i:Person_hasInterest_Tag]->
    (t:Tag WHERE t.id = 6)-[i2:Person_hasInterest_Tag]-(p:Person)
    COLUMNS (p.firstName as p1name, p2.firstName as p2name) 
    ) pg
    ORDER BY pg.p1name, pg.p2name


In [ ]:
%%sql
    SELECT pg.bob, pg.bob_id, count(pg.friend_name) as number_of_friends

    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person WHERE p.firstName = 'Bob')-[k:Person_knows_Person]->(p2:person)
    COLUMNS (p.firstName as bob, p.id as bob_id, p2.firstName as friend_name) 
    ) pg

    GROUP BY pg.bob, pg.bob_id